# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [50]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [51]:
#Import CSV and create data frame
cities_data_path = "../output_data/cities.csv"
cities_data = pd.read_csv(cities_data_path)
cities_data_df = pd.DataFrame(cities_data)
#Drop prior index column
cities_data_df.drop(['Unnamed: 0'], inplace= True, axis = 1)
#Display data frame preview
cities_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,pisco,-13.7000,-76.2167,75.20,64,0,8.05,PE,1611771279
1,saldanha,-33.0117,17.9442,66.20,59,0,20.71,ZA,1611771280
2,kapaa,22.0752,-159.3190,73.40,88,90,13.80,US,1611770983
3,port elizabeth,-33.9180,25.5701,66.20,94,75,6.91,ZA,1611771280
4,atuona,-9.8000,-139.0333,78.13,77,4,15.35,PF,1611771280


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [52]:
#Create new column for lat,lon tuples for gmaps
cities_data_df['Lat, Lng'] = list(zip(cities_data_df.Lat, cities_data_df.Lng))
#Configure gmaps
gmaps.configure(g_key)
fig = gmaps.figure(center=(35, -20), zoom_level=2)
locations = cities_data_df['Lat, Lng']
weights = cities_data_df['Humidity']
#Add heat map layer
heatmap = gmaps.heatmap_layer(locations, weights=weights, point_radius = 15)
heatmap.max_intensity = 100
fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [54]:
#Create filters for ideal weather cities: max temp between 70-80, wind speed less than 5 mph, zero cloudiness
filter1 = cities_data_df['Max Temp']>70
filter2 = cities_data_df['Max Temp']<80
filter3 = cities_data_df['Wind Speed']<10
filter4 = cities_data_df['Cloudiness']==0
#Create filtered data frame and display
ideal_weather_df = cities_data_df[filter1 & filter2 & filter3 & filter4]
ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,"Lat, Lng"
0,pisco,-13.7000,-76.2167,75.20,64,0,8.05,PE,1611771279,"(-13.7, -76.2167)"
176,salalah,17.0151,54.0924,75.20,69,0,3.44,OM,1611771334,"(17.0151, 54.0924)"
235,sao filipe,14.8961,-24.4956,76.93,63,0,5.59,CV,1611771350,"(14.8961, -24.4956)"
343,berbera,10.4396,45.0143,74.75,71,0,8.52,SO,1611771385,"(10.4396, 45.0143)"
351,djibo,14.1022,-1.6306,79.63,12,0,8.05,BF,1611771388,"(14.1022, -1.6306)"
493,aguimes,27.9054,-15.4461,71.01,83,0,6.91,ES,1611771419,"(27.9054, -15.4461)"
529,necochea,-38.5473,-58.7368,73.00,42,0,5.99,AR,1611771433,"(-38.5473, -58.7368)"
539,dakar,14.6937,-17.4441,75.20,46,0,9.22,SN,1611771234,"(14.6937, -17.4441)"


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [61]:
#Create hotel_df and add hotel name column
hotel_df = ideal_weather_df
hotel_df['Hotel Name']=''
hotel_df


C:\Users\Noah\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,"Lat, Lng",Hotel Name
0,pisco,-13.7000,-76.2167,75.20,64,0,8.05,PE,1611771279,"(-13.7, -76.2167)",
176,salalah,17.0151,54.0924,75.20,69,0,3.44,OM,1611771334,"(17.0151, 54.0924)",
235,sao filipe,14.8961,-24.4956,76.93,63,0,5.59,CV,1611771350,"(14.8961, -24.4956)",
343,berbera,10.4396,45.0143,74.75,71,0,8.52,SO,1611771385,"(10.4396, 45.0143)",
351,djibo,14.1022,-1.6306,79.63,12,0,8.05,BF,1611771388,"(14.1022, -1.6306)",
493,aguimes,27.9054,-15.4461,71.01,83,0,6.91,ES,1611771419,"(27.9054, -15.4461)",
529,necochea,-38.5473,-58.7368,73.00,42,0,5.99,AR,1611771433,"(-38.5473, -58.7368)",
539,dakar,14.6937,-17.4441,75.20,46,0,9.22,SN,1611771234,"(14.6937, -17.4441)",


In [57]:
#Create base URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#Create params
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

#  use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    #get lat lng from hotel_df
    lat = row['Lat']
    lng = row['Lng']
    #add keyword to params dict
    params['location'] = f"{lat},{lng}"
    #locate city for print request output
    city = row['City']
    #assemble url and make API request
    response = requests.get(base_url, params=params).json()
    print(f"Retrieving Results for Index {index}: {city}")
     #extract results
    results = response['results']
    #Use try/except to check and find hotel name and put name into hotel data frame
    try:
        print(f"Closest {city} hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


Retrieving Results for Index 0: pisco
Closest pisco hotel is Hotel San Isidro Oficial.


C:\Users\Noah\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Retrieving Results for Index 176: salalah
Closest salalah hotel is HAMDAN PLAZA HOTEL SALALAH.
Retrieving Results for Index 235: sao filipe
Closest sao filipe hotel is Tortuga B&B.
Retrieving Results for Index 343: berbera
Closest berbera hotel is IBCO REAL ESTATE PROPERTY.
Retrieving Results for Index 351: djibo
Closest djibo hotel is Centre Des Ressources DPENA.
Retrieving Results for Index 493: aguimes
Closest aguimes hotel is Hotel Villa de Aguimes.
Retrieving Results for Index 529: necochea
Closest necochea hotel is dyd -mar Hotel.
Retrieving Results for Index 539: dakar
Closest dakar hotel is Novotel Dakar.


In [58]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,"Lat, Lng",Hotel Name
0,pisco,-13.7000,-76.2167,75.20,64,0,8.05,PE,1611771279,"(-13.7, -76.2167)",Hotel San Isidro Oficial
176,salalah,17.0151,54.0924,75.20,69,0,3.44,OM,1611771334,"(17.0151, 54.0924)",HAMDAN PLAZA HOTEL SALALAH
235,sao filipe,14.8961,-24.4956,76.93,63,0,5.59,CV,1611771350,"(14.8961, -24.4956)",Tortuga B&B
343,berbera,10.4396,45.0143,74.75,71,0,8.52,SO,1611771385,"(10.4396, 45.0143)",IBCO REAL ESTATE PROPERTY
351,djibo,14.1022,-1.6306,79.63,12,0,8.05,BF,1611771388,"(14.1022, -1.6306)",Centre Des Ressources DPENA
493,aguimes,27.9054,-15.4461,71.01,83,0,6.91,ES,1611771419,"(27.9054, -15.4461)",Hotel Villa de Aguimes
529,necochea,-38.5473,-58.7368,73.00,42,0,5.99,AR,1611771433,"(-38.5473, -58.7368)",dyd -mar Hotel
539,dakar,14.6937,-17.4441,75.20,46,0,9.22,SN,1611771234,"(14.6937, -17.4441)",Novotel Dakar


In [59]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [60]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
markers= gmaps.marker_layer(locations, hotel_info)
fig.add_layer(markers)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))